### Creating the SparkContext and SparkSession objects to connect to the Spark Cluster

You will need to "connect" to the cluster in each Jupyter notebook to run the code.

The first step is finding the necessary Spark related environment variables using the following cell:

In [1]:
import findspark
findspark.init()

You have to import the `pyspark` module, and to connect to the cluster in this workbook you can run the following cell. 

You may want to copy the previous cell to other workbooks you start, and you can change the `appName` to a relevant name for your work.

In [2]:
from pyspark import SparkContext, SparkConf, HiveContext
from pyspark.sql import SparkSession, SQLContext

spark = SparkSession.builder.appName("lab-06-app").getOrCreate() # get the spark session object

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/10/12 03:10:26 WARN Client: Neither spark.yarn.jars nor spark.yarn.archive is set, falling back to uploading libraries under SPARK_HOME.
22/10/12 03:10:36 WARN YarnSchedulerBackend$YarnSchedulerEndpoint: Attempted to request executors before the AM has registered!


The previous cell creates the essential spark object that you will use to interact with the cluster and direct operations.

In [3]:
# old contexts that are here for backwards compatibility
sc           = spark.sparkContext
sql_context  = SQLContext(sc)
hive_context = HiveContext(sc)

The previous cell shows you other ways in which you can interact with the cluster. You will not need these for the purposes of this class, but code in version PySpark < 3 will use spark context `sc` instead of the `spark` variable. In PySpark 3+ you can use the spark context and spark session interchangably.

### **Important!: Close the spark session before you close your notebook.**

Just as you have to create the spark session object, you need to shut down the session before you exit the Jupyter notebook. If you do not do this, then you will have an abandoned session to the cluster which will have asked for resources and not released them. Remember, Spark uses memory on the cluster, so if the memorys already allocated to another application when you create a spark session object, then Spark will not perform.

Closing the session is very easy. (Do not run the following cell unless you want to close the session). 

This cell has not been added to the example notebooks, it is up to you do do it.

In [9]:
spark.stop()

If you ran the previous cell and want you to run the cells below this, then comment out the cell code and restart your kernel to make a new spark session and not close it.

### Explanation of SparkContext and SparkSession

** Note: This section was taken verbatim from the book [Fast Data Processing with Spark - 3ed](https://www.packtpub.com/big-data-and-business-intelligence/fast-data-processing-spark-2-third-edition).**

Prior to Spark 2.0.0, the three main connection objects were `SparkContext, SqlContext, and HiveContext`. The `SparkContext` object was the connection to a Spark execution environment and created RDDs and others, `SQLContext` worked with `SparkSQL` in the background of `SparkContext`, and `HiveContext` interacted with the Hive stores.

Spark 2.0.0 introduced Datasets/DataFrames as the main distributed data abstraction interface and the `SparkSession` object as the entry point to a Spark execution environment. Appropriately, the `SparkSession` object is found in the namespace, `org.apache.spark.sql.SparkSession` (Scala), or `pyspark.sql.sparkSession`. A few points to note are as follows:

* In Scala and Java, Datasets form the main data abstraction as typed data; however, for Python and R (which do not have compile time type checking), the data abstraction is DataFrame. For all practical API purposes, the Datasets in Scala/Java are the same as DataFrames in Python/R.
* While Datasets/DataFrames are top-level interfaces, RDDs have not disappeared. In fact, the underlying structures are still RRDs. Also, to interact with RDDs, we still need a SparkContext object and we can get one from the SparkSession object.
* The `SparkSession` object encapsulates the `SparkContext` object. As of version 2.0.0, `SparkContext` is still the conduit to a Spark cluster (local or remote)

However, for operations such as reading and creating Datasets, use the `SparkSession` object.

To see some of the attributes associated with the `SparkSession` and `SparkContext` you can run some methods:

In [4]:
sc.version

'3.0.0-amzn-0'

In [5]:
spark.version

'3.0.0-amzn-0'

You can see that your cluster is connected to a cluster using YARN

In [6]:
sc.master

'yarn'

You can check the standard configuration when spinning up spark by running the following commands

In [7]:
from pyspark.conf import SparkConf
conf = SparkConf()
print(conf.toDebugString())

spark.app.name=lab-06-app
spark.blacklist.decommissioning.enabled=true
spark.blacklist.decommissioning.timeout=1h
spark.decommissioning.timeout.threshold=20
spark.default.parallelism=16
spark.driver.defaultJavaOptions=-XX:OnOutOfMemoryError='kill -9 %p' -XX:+UseConcMarkSweepGC -XX:CMSInitiatingOccupancyFraction=70 -XX:MaxHeapFreeRatio=70 -XX:+CMSClassUnloadingEnabled
spark.driver.extraClassPath=/usr/lib/hadoop-lzo/lib/*:/usr/lib/hadoop/hadoop-aws.jar:/usr/share/aws/aws-java-sdk/*:/usr/share/aws/emr/emrfs/conf:/usr/share/aws/emr/emrfs/lib/*:/usr/share/aws/emr/emrfs/auxlib/*:/usr/share/aws/emr/goodies/lib/emr-spark-goodies.jar:/usr/share/aws/emr/security/conf:/usr/share/aws/emr/security/lib/*:/usr/share/aws/hmclient/lib/aws-glue-datacatalog-spark-client.jar:/usr/share/java/Hive-JSON-Serde/hive-openx-serde.jar:/usr/share/aws/sagemaker-spark-sdk/lib/sagemaker-spark-sdk.jar:/usr/share/aws/emr/s3select/lib/emr-s3-select-spark-connector.jar:/docker/usr/lib/hadoop-lzo/lib/*:/docker/usr/lib/had

You can examine the current configuration for your spark application by running the following cell

In [8]:
spark.sparkContext.getConf().getAll()

[('spark.eventLog.enabled', 'true'),
 ('spark.hadoop.mapreduce.fileoutputcommitter.algorithm.version.emr_internal_use_only.EmrFileSystem',
  '2'),
 ('spark.org.apache.hadoop.yarn.server.webproxy.amfilter.AmIpFilter.param.PROXY_URI_BASES',
  'http://ip-172-31-16-60.ec2.internal:20888/proxy/application_1665542883724_0001'),
 ('spark.sql.parquet.output.committer.class',
  'com.amazon.emr.committer.EmrOptimizedSparkSqlParquetOutputCommitter'),
 ('spark.driver.extraClassPath',
  '/usr/lib/hadoop-lzo/lib/*:/usr/lib/hadoop/hadoop-aws.jar:/usr/share/aws/aws-java-sdk/*:/usr/share/aws/emr/emrfs/conf:/usr/share/aws/emr/emrfs/lib/*:/usr/share/aws/emr/emrfs/auxlib/*:/usr/share/aws/emr/goodies/lib/emr-spark-goodies.jar:/usr/share/aws/emr/security/conf:/usr/share/aws/emr/security/lib/*:/usr/share/aws/hmclient/lib/aws-glue-datacatalog-spark-client.jar:/usr/share/java/Hive-JSON-Serde/hive-openx-serde.jar:/usr/share/aws/sagemaker-spark-sdk/lib/sagemaker-spark-sdk.jar:/usr/share/aws/emr/s3select/lib/emr-